In [ ]:
import sys, os
cwd=os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, cwd)
import rasterio
import folium
import branca
from folium.plugins import HeatMap
import re
import branca.colormap as cmp
import xgboost as xgb
import pandas as pd
import geopandas as gpd
import shapefile as shp
from shapely.geometry import Point
from shapely.geometry.polygon import Point, Polygon
import matplotlib.pyplot as plt
import rioxarray as rxr
import xarray as xr
import numpy as np
from scipy.spatial.distance import cdist
import utils.processing as pr
import utils.s3_utils as s3
pd.set_option('display.max_columns', None)
from sklearn.preprocessing import PolynomialFeatures
from folium import plugins
import config.paths as path
import xgboost as xgb
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, MinMaxScaler, RobustScaler
from sklearn.model_selection import KFold, GridSearchCV
from pathlib import Path

region_code_list = [0,1,2]
modelname = path.model_path

In [ ]:

for region_code in region_code_list: 
    #get regional training data
    trainingdata_path = f'../../data/{modelname}/training/final/clustered/region_{region_code}/'
    fn = trainingdata_path + f'training_data/{region_code}-training-data.csv'
    df_foci = pd.read_csv(fn).reset_index(drop=True)
    df_foci = df_foci.drop('Unnamed: 0',axis=1)

    # Renaming columns
    df_foci  = df_foci.rename(columns={'longitude_left': 'longitude', 'latitude_left': 'latitude'})
    df_foci.columns = df_foci.columns.str.lower()
    df_foci = df_foci.rename({
                'tg-grp-mean-days-above-5degc-monthly-ratio':'tg-grp-mean-days-above-5degC-monthly-ratio', 
                }, axis=1)
    display(df_foci.shape)
    df1 = df_foci

    # get the covariate values and labels given in config.paths to be consistent with maxent 2.4 processing
    # also filter by the columns given in the config.paths
    cov_list = list(path.cov_dict.keys())[1:]
    cov_list.append('nuts_id')
    cov_list.append('nuts_name')
    cov_list.append('latitude')
    cov_list.append('longitude')
    df1 = df1[cov_list]

    df1['presence'] = df_foci['presence']
    df1['cat'] = df_foci['cat']
    inv_cov_dict = {v: k for k, v in path.cov_dict.items()}
    df1 = df1.rename(columns = inv_cov_dict)

    df1_presence = df1[(df1.presence == 1)]
    display(df1.shape)

    print('Exporting x and y foci-only data... ')
    df1.to_csv(trainingdata_path + 'training-foci-only/x.csv')

    #append pseudoabsence data
    #get regional pseudoabsence data
    fn = trainingdata_path + f'pseudoabsence_data/{region_code}-pseudoabsence-data.csv'
    df_pseudoabsence = pd.read_csv(fn).reset_index(drop=True)
    df_pseudoabsence = df_pseudoabsence.drop('Unnamed: 0',axis=1)



    # get the covariate values and labels given in config.paths to be consistent with maxent 2.4 processing
    # also filter by the columns given in the config.paths
    # also include nuts_id, nuts_name
    cov_list = list(path.cov_dict.keys())[1:]
    cov_list.append('nuts_id')
    cov_list.append('nuts_name')
    cov_list.append('lon_env')
    cov_list.append('lat_env')
    pseudo_df = df_pseudoabsence[cov_list]

    pseudo_df  = pseudo_df.rename(columns={'lon_env': 'longitude', 'lat_env': 'latitude'})


    pseudo_df['presence'] = df_pseudoabsence['presence']
    pseudo_df['cat'] = df_pseudoabsence['cat']
    inv_cov_dict = {v: k for k, v in path.cov_dict.items()}
    pseudo_df = pseudo_df.rename(columns = inv_cov_dict)


    #drop unnecessary columns
    x_train = df1_presence.drop_duplicates()

    # Pseudoabsence filtered modeling
    print(f'Check to see if pseudoabsence covariates line up with training_dataset: {set(pseudo_df.columns).difference(set(x_train.columns))}')

    x_train = x_train[x_train.columns.sort_values()]
    pseudo_df = pseudo_df[pseudo_df.columns.sort_values()]

    #concatenate tbev data with pseudoabsence points
    x_train = pd.concat([x_train,pseudo_df])

    # #get dummies for landcover 
    dummies_train = pd.get_dummies(x_train[['cat']],dummy_na=True).drop('cat_nan',axis=1)
    x_train = x_train.drop(['cat','landcover'],axis=1)

    x_train = pd.concat([x_train,dummies_train],axis=1).reset_index(drop=True)


    y = x_train[['presence','nuts_id','nuts_name','latitude','longitude']]
    x = x_train.drop(['presence'],axis=1)

    print('Exporting x and y data as csvs... ')
    x.to_csv(trainingdata_path + 'training_x/x.csv')
    y.to_csv(trainingdata_path + 'training_y/y.csv')
    dummies_train.to_csv(trainingdata_path + 'training_x/dummies.csv')

    import scipy
    import scipy
    import pandas as pd
    region = 0

    p_val = .01
    corr_val=.95

    landcover_cols = [col for col in x.columns if 'cat_' in col]
    X = x.drop(['nuts_id','nuts_name','latitude','longitude'],axis=1)
    X = X.drop(landcover_cols,axis=1)

    #Calculate Pearson correlation coefficient and p-value
    cov_pair_list = []
    pos_covs=[]
    corr_var_text = f'*** Pearson Correlation Coef. Related Variables (p<{p_val}, corr>{corr_val}) ***'
    for cov1 in X.columns:
        for cov2 in X.columns:
            if cov1==cov2: #excludes self-correlating the same cov
                pass
            elif (cov1, cov2) in cov_pair_list: #avoids duplicate correlations (cov1,cov2) == (cov2,cov1)
                pass
            else:
                cov_pair_list.append((cov1,cov2))
                corr, p_value = scipy.stats.pearsonr(X[cov1], X[cov2])

                # Interpretation of the correlation
                if (p_value < p_val) & (corr > corr_val):
                    print(f"Results for {cov1}, {cov2}")
                    print(f"Pearson Correlation Coefficient: {corr}")
                    print(f"P-value: {p_value:.9f}")
                    print ('***********************************')
                    corr_var_text = ''.join((corr_var_text,'\n' + f'{cov1} and {cov2} are related',''))
                    pos_covs.append((cov2,cov1))
                else:
                    pass
            cov_pair_list.append((cov2,cov1))
    print(corr_var_text)
    r_ptest_corr_df = pd.DataFrame({'var1':[x[0] for x in pos_covs], 'var2':[x[1] for x in pos_covs]})
    r_ptest_corr_df.to_csv(trainingdata_path + "stats_tests/pearson-ptest-correlation-results.csv")



In [ ]:
#output foci only data incl absences
df1.to_csv(trainingdata_path + 'training-foci-only/foci.csv')

In [ ]:
#append pseudoabsence data
#get regional pseudoabsence data
fn = trainingdata_path + f'pseudoabsence_data/{region_code}-pseudoabsence-data.csv'
df_pseudoabsence = pd.read_csv(fn).reset_index(drop=True)
df_pseudoabsence = df_pseudoabsence.drop('Unnamed: 0',axis=1)


# get the covariate values and labels given in config.paths to be consistent with maxent 2.4 processing
# also filter by the columns given in the config.paths
# also include nuts_id, nuts_name
cov_list = list(path.cov_dict.keys())[1:]
cov_list.append('nuts_id')
cov_list.append('nuts_name')
cov_list.append('lon_env')
cov_list.append('lat_env')
pseudo_df = df_pseudoabsence[cov_list]

pseudo_df  = pseudo_df.rename(columns={'lon_env': 'longitude', 'lat_env': 'latitude'})


pseudo_df['presence'] = df_pseudoabsence['presence']
pseudo_df['cat'] = df_pseudoabsence['cat']
inv_cov_dict = {v: k for k, v in path.cov_dict.items()}
pseudo_df = pseudo_df.rename(columns = inv_cov_dict)

print(pseudo_df.columns)
print(df1_presence.columns) 


In [ ]:
#drop unnecessary columns
x_train = df1_presence.drop_duplicates()

# Pseudoabsence filtered modeling
print(f'Check to see if pseudoabsence covariates line up with training_dataset: {set(pseudo_df.columns).difference(set(x_train.columns))}')

x_train = x_train[x_train.columns.sort_values()]
pseudo_df = pseudo_df[pseudo_df.columns.sort_values()]

#concatenate tbev data with pseudoabsence points
x_train = pd.concat([x_train,pseudo_df])

# #get dummies for landcover 
dummies_train = pd.get_dummies(x_train[['cat']],dummy_na=True).drop('cat_nan',axis=1)
x_train = x_train.drop(['cat','landcover'],axis=1)

x_train = pd.concat([x_train,dummies_train],axis=1).reset_index(drop=True)


y = x_train[['presence','nuts_id','nuts_name','latitude','longitude']]
x = x_train.drop(['presence'],axis=1)


In [ ]:
x.to_csv(trainingdata_path + 'training_x/x.csv')
y.to_csv(trainingdata_path + 'training_y/y.csv')
dummies_train.to_csv(trainingdata_path + 'training_x/dummies.csv')

## Variable Selection - Pearson Method

In [ ]:
# Function to extract related variables based on a threshold
def extract_related_variables(correlation_matrix_df, threshold=0.7):
    related_variables = []
    n = correlation_matrix_df.shape[0]
    for i in range(n):
        for j in range(i+1, n):
            if abs(correlation_matrix_df.iloc[i, j]) >= threshold:
                related_variables.append((correlation_matrix_df.columns[i], correlation_matrix_df.columns[j]))
    return related_variables

In [ ]:
#Get pearson correlation matrix using x but drop nuts district cols.
corr_df = x.drop(['nuts_id','nuts_name','latitude','longitude'],axis=1).corr(method='pearson')
corr_df.to_csv(trainingdata_path+'stats_tests/pearson_matrix.csv')

# Set your desired threshold
pearson_threshold=.90

corr_df = corr_df[corr_df!=1]

# Extract related variables
related_variables = extract_related_variables(corr_df, pearson_threshold)

# Print the related variables
corr_var_text = '*** Pearson Correlation Coef. Related Variables ***'
for var1, var2 in related_variables:
    corr_var_text = ''.join((corr_var_text,'\n' + f'{var1} and {var2} are related',''))

print(corr_var_text)

# Save related variables as text file
with open(trainingdata_path+"stats_tests/pearson-correlation-results.txt", "w") as text_file:
   text_file.write(corr_var_text)


## Using scipy (deprecated)


In [ ]:
import scipy
import scipy
import pandas as pd
region = 0

p_val = .01
corr_val=.95

landcover_cols = [col for col in x.columns if 'cat_' in col]
X = x.drop(['nuts_id','nuts_name','latitude','longitude'],axis=1)
X = X.drop(landcover_cols,axis=1)

#Calculate Pearson correlation coefficient and p-value
cov_pair_list = []
pos_covs=[]
corr_var_text = f'*** Pearson Correlation Coef. Related Variables (p<{p_val}, corr>{corr_val}) ***'
for cov1 in X.columns:
    for cov2 in X.columns:
        if cov1==cov2: #excludes self-correlating the same cov
            pass
        elif (cov1, cov2) in cov_pair_list: #avoids duplicate correlations (cov1,cov2) == (cov2,cov1)
            pass
        else:
            cov_pair_list.append((cov1,cov2))
            corr, p_value = scipy.stats.pearsonr(X[cov1], X[cov2])

            # Interpretation of the correlation
            if (p_value < p_val) & (corr > corr_val):
                print(f"Results for {cov1}, {cov2}")
                print(f"Pearson Correlation Coefficient: {corr}")
                print(f"P-value: {p_value:.9f}")
                print ('***********************************')
                corr_var_text = ''.join((corr_var_text,'\n' + f'{cov1} and {cov2} are related',''))
                pos_covs.append((cov2,cov1))
            else:
                pass
        cov_pair_list.append((cov2,cov1))
print(corr_var_text)
r_ptest_corr_df = pd.DataFrame({'var1':[x[0] for x in pos_covs], 'var2':[x[1] for x in pos_covs]})
r_ptest_corr_df.to_csv(trainingdata_path + "stats_tests/pearson-ptest-correlation-results.csv")


In [ ]:
r_ptest_corr_df = pd.DataFrame({'var1':[x[0] for x in pos_covs], 'var2':[x[1] for x in pos_covs]})
r_ptest_corr_df